<a href="https://colab.research.google.com/github/Narawit007/dataviz2025/blob/main/midterm2025_663020283_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

## เตรียมนำเข้าข้อมูล

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pandas as pd

BASE = "/content/drive/MyDrive/midthem"

# โหลดไฟล์ทั้งหมด (บังคับ dtype=str เพื่อลดปัญหาศูนย์นำหน้า/ชนิดข้อมูล)
read_opts = dict(dtype=str)

csr  = pd.read_csv(f"{BASE}/crime_scene_report.csv", **read_opts)
pers = pd.read_csv(f"{BASE}/person.csv", **read_opts)                 # id, name, license_id, address_number, address_street_name, ssn
dl   = pd.read_csv(f"{BASE}/drivers_license.csv", **read_opts)        # id, age, height, eye_color, hair_color, gender, plate_number, car_make, car_model
iv   = pd.read_csv(f"{BASE}/interview.csv", **read_opts)              # person_id, transcript
memb = pd.read_csv(f"{BASE}/get_fit_now_member.csv", **read_opts)     # id, person_id, name, membership_start_date, membership_status
cin  = pd.read_csv(f"{BASE}/get_fit_now_check_in.csv", **read_opts)   # membership_id, check_in_date, check_in_time, check_out_time
fb   = pd.read_csv(f"{BASE}/facebook_event_checkin.csv", **read_opts) # person_id, event_id, event_name, date
inc  = pd.read_csv(f"{BASE}/income.csv", **read_opts)                 # ssn, annual_income

# ช่วย: to_int (แปลงเลขที่เก็บเป็นสตริงให้เป็น int แบบปลอดภัย)
def to_int(s):
    return pd.to_numeric(s, errors="coerce").astype("Int64")


## ดึงบันทึกคดี

In [15]:
# 1) ดึงบันทึกคดี: murder @ Pandas City วันที่ 2018-01-15
csr["date_int"] = to_int(csr.get("date"))
mask = (
    csr.get("type", "").str.contains("murder", case=False, na=False) &
    csr.get("city", "").str.contains("Pandas City", case=False, na=False) &
    (csr["date_int"] == 20180115)
)
crime_rows = csr[mask].copy()
assert len(crime_rows) >= 1, "ไม่พบแถวคดี murder ที่ Pandas City วันที่ 2018-01-15 ใน crime_scene_report.csv"

crime_desc = crime_rows.iloc[0].get("description", "")
print("Crime scene description:\n", crime_desc, "\n")

Crime scene description:
 Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave". 



## ระบุพยานในคดี



In [16]:
# 2) พยาน:
# - พยาน 1: "last house on Northwestern Dr" -> เลขที่บ้านสูงสุดบนถนนนี้
# - พยาน 2: "Annabel" บน "Franklin Ave"
pers["address_number_int"] = to_int(pers.get("address_number"))

w1_candidates = pers[pers.get("address_street_name", "").str.contains("Northwestern Dr", case=False, na=False)]
assert len(w1_candidates) > 0, "ไม่พบผู้อยู่อาศัยบน Northwestern Dr"
w1 = w1_candidates.loc[w1_candidates["address_number_int"].idxmax()]

w2_candidates = pers[
    pers.get("name", "").str.contains(r"\bAnnabel\b", case=False, na=False) &
    pers.get("address_street_name", "").str.contains("Franklin Ave", case=False, na=False)
]
assert len(w2_candidates) > 0, "ไม่พบ Annabel บน Franklin Ave"
w2 = w2_candidates.iloc[0]

print(f"Witness 1: {w1['name']} (id={w1['id']}) @ {w1['address_number']} {w1['address_street_name']}")
print(f"Witness 2: {w2['name']} (id={w2['id']}) @ {w2['address_number']} {w2['address_street_name']}\n")

Witness 1: Morty Schapiro (id=14887) @ 4919 Northwestern Dr
Witness 2: Annabel Miller (id=16371) @ 103 Franklin Ave



## คำให้การพยาน

In [17]:
# 3) อ่านปากคำพยาน -> เงื่อนงำ: ถุง 'Get Fit Now', ทะเบียนมี 'H42W', เจอที่ยิม 2018-01-09
wit_iv = iv[iv["person_id"].isin([w1["id"], w2["id"]])].copy()
assert len(wit_iv) >= 2, "ไม่พบคำให้การของพยานทั้งสอง"
for _, r in wit_iv.iterrows():
    print(f"Interview person_id={r['person_id']}:\n{r['transcript']}\n")

Interview person_id=14887:
I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

Interview person_id=16371:
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.



## เบอะแสจากพยาน

In [18]:
# 4) กรองเช็คอินยิมวันที่ 2018-01-09 แล้วดูสมาชิกที่น่าสงสัย
cin["check_in_date_int"] = to_int(cin["check_in_date"])
cin_0109 = cin[cin["check_in_date_int"] == 20180109].copy()
assert len(cin_0109) > 0, "ไม่พบเช็คอิน 2018-01-09 ใน get_fit_now_check_in.csv"

sus_member_ids = sorted(set([mid for mid in cin_0109["membership_id"].dropna() if str(mid).startswith("48Z")]))
print("Candidate member_ids on 2018-01-09 starting with '48Z':", sus_member_ids)
assert len(sus_member_ids) >= 1, "ไม่พบสมาชิกขึ้นต้น 48Z ในวัน 2018-01-09"

Candidate member_ids on 2018-01-09 starting with '48Z': ['48Z55', '48Z7A']


## สืบหาผู้ต้องสงสัยในgym

In [20]:
# 5) จับคู่สมาชิก -> บุคคล -> ใบขับขี่ แล้วหา plate_number มี 'H42W'
memb_renamed = memb.rename(columns={"name": "member_name"})
sus_m = memb_renamed[memb_renamed["id"].isin(sus_member_ids)].copy()
assert len(sus_m) >= 1, "ไม่พบข้อมูลสมาชิกที่สอดคล้อง"

sus = (sus_m
       .merge(pers.rename(columns={"name": "person_name"}), left_on="person_id", right_on="id", how="left", suffixes=("", "_pers"))
       .merge(dl, left_on="license_id", right_on="id", how="left", suffixes=("", "_dl")))
assert "plate_number" in sus.columns, "ไม่มีคอลัมน์ plate_number หลัง merge กับ drivers_license.csv"
sus["has_H42W"] = sus["plate_number"].astype(str).str.contains("H42W", na=False)
murderer_row = sus[sus["has_H42W"]].copy()
assert len(murderer_row) >= 1, "ไม่พบผู้ต้องสงสัยที่ทะเบียนมี 'H42W'"

murderer_row = murderer_row.iloc[0]
murderer_name = murderer_row["person_name"] if "person_name" in murderer_row else murderer_row.get("member_name", "UNKNOWN")
print(f"\n[RESULT] ฆาตกร: {murderer_name} "
      f"(member_id={murderer_row['id']}, plate={murderer_row['plate_number']})")


[RESULT] ฆาตกร: Jeremy Bowers (member_id=48Z55, plate=0H42W2)


##

## สิ่งที่ได้จากคำให้การของฆาตกร

In [21]:
# 6) อ่านปากคำฆาตกร -> เบาะแสผู้บงการ: ผู้หญิงรวย / ไป SQL Symphony Concert 3 ครั้งใน Dec 2017
murderer_pid = murderer_row["person_id"]
jer_iv = iv[iv["person_id"] == murderer_pid]
assert len(jer_iv) >= 1, "ไม่พบคำให้การของฆาตกร"
print("\nMurderer interview:\n", jer_iv.iloc[0]["transcript"], "\n")


Murderer interview:
 I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.
 



##

## หาคนที่ตรงกับคำให้การของฆาตกร

In [22]:
# 7) หา 'SQL Symphony Concert' ในเดือน 2017-12 ทั้งเดือน และนับคนที่ไป 3 ครั้ง
fb["date_int"] = to_int(fb["date"])
fb_dec = fb[
    fb.get("event_name", "").str.contains("SQL Symphony Concert", case=False, na=False) &
    fb["date_int"].between(20171201, 20171231)
].copy()
counts = fb_dec.groupby("person_id").size().reset_index(name="count")
three_times = counts[counts["count"] == 3]
assert len(three_times) >= 1, "ไม่พบผู้ที่ไป SQL Symphony Concert จำนวน 3 ครั้งใน Dec 2017"

## ระบุผู้บงการ

In [23]:
# 8) ระบุตัวผู้บงการ: จับคู่กับ person -> drivers_license -> income
master = (three_times
          .merge(pers.rename(columns={"name": "person_name"}), left_on="person_id", right_on="id", how="left")
          .merge(dl, left_on="license_id", right_on="id", how="left", suffixes=("", "_dl"))
          .merge(inc, on="ssn", how="left"))
# เลือกคนรายได้มากสุด เผื่อกรณีได้หลายคน
master["annual_income_int"] = to_int(master["annual_income"])
master = master.sort_values("annual_income_int", ascending=False).iloc[0]

master_name = master["person_name"]
print(f"[RESULT] ผู้บงการ: {master_name} "
      f"(income={master.get('annual_income')}, hair={master.get('hair_color')}, "
      f"height={master.get('height')}, car={master.get('car_make')} {master.get('car_model')})")

[RESULT] ผู้บงการ: Miranda Priestly (income=310000, hair=red, height=66, car=Tesla Model S)


## สรุปคำตอบที่ต้องการ

In [24]:
# 9) สรุปย่อ
print("\n=== SUMMARY ===")
print("- พยาน: ถุง 'Get Fit Now' (มันคือกระเป๋าสมาชิก Gold ของฟิตเนส Get Fit Now Gym)', ทะเบียนมี 'H42W', เคยเห็นที่ยิมวันที่ 2018-01-09")
print(f"- ฆาตกร: {murderer_name} (member_id={murderer_row['id']}, plate={murderer_row['plate_number']})")
print(f"- ผู้บงการ: {master_name} (รายได้สูง/ไป SQL Symphony Concert 3 ครั้งใน Dec 2017)")


=== SUMMARY ===
- พยาน: ถุง 'Get Fit Now' (มันคือกระเป๋าสมาชิก Gold ของฟิตเนส Get Fit Now Gym)', ทะเบียนมี 'H42W', เคยเห็นที่ยิมวันที่ 2018-01-09
- ฆาตกร: Jeremy Bowers (member_id=48Z55, plate=0H42W2)
- ผู้บงการ: Miranda Priestly (รายได้สูง/ไป SQL Symphony Concert 3 ครั้งใน Dec 2017)


# สรุปเรื่องที่เกิดขึ้นทั้งหมด

ในวันที่ 15 มกราคม 2018 เมือง Pandas City เกิดคดีฆาตกรรมขึ้น ตำรวจได้เบาะแสจาก สองพยานในคดี

พยานคนแรก: อาศัยอยู่บ้านหลังสุดท้ายบนถนน Northwestern Dr
- เขาได้ยินเสียงปืน แล้วเห็นผู้ชายวิ่งหนีออกมา พร้อมถือ ถุงยิม Get Fit Now อยู่ในมือ
- ที่กระเป๋ามีเลขสมาชิกขึ้นต้นด้วย 48Z ซึ่งเป็นรหัสของสมาชิก Gold เท่านั้น
- จากนั้นชายคนนั้นก็ขึ้นรถที่ป้ายทะเบียนมีตัวอักษร H42W


พยานคนที่สอง: ชื่อ Annabel Miller อาศัยบนถนน Franklin Ave
- เธอบอกว่าเธอจำคนร้ายได้ เพราะเคยเห็นเขาที่ฟิตเนสเมื่อวันที่ 9 มกราคม 2018 (ไม่กี่วันก่อนเกิดเหตุ)

ตำรวจตรวจสอบบันทึกเช็คอินยิม Get Fit Now วันที่ 9 ม.ค. 2018
- พบผู้ต้องสงสัยที่มีรหัสขึ้นต้น 48Z อยู่ 2 คน: Joe Germuska และ Jeremy Bowers

เมื่อตรวจสอบทะเบียนรถในระบบ พบว่า Jeremy Bowers ใช้รถที่ป้ายทะเบียน 0H42W2 ซึ่งมี “H42W” อยู่ตรงกลาง
- จึงยืนยันได้ว่า Jeremy Bowers คือฆาตกร

เมื่อสอบปากคำ Jeremy เขาสารภาพว่า เขาไม่ได้ทำเพราะความแค้นส่วนตัว แต่ถูกจ้าง
- ผู้ที่จ้างเป็น ผู้หญิงคนหนึ่ง เธอมีเงินมาก, ผมสีแดง, ขับ Tesla Model S, สูงราว 5.5–5.7ฟุต (ประมาณ167-173cmมั้งคิดว่า) และในเดือนธันวาคม 2017 เธอไปงาน SQL Symphony Concert ถึง 3 ครั้ง

ตำรวจจึงตรวจสอบบันทึกการเช็คอินงานคอนเสิร์ต พบว่า มีผู้หญิงเพียงคนเดียวที่เข้าเงื่อนไขทุกข้อ คือ Miranda Priestly
- เธอมีรายได้สูง (310,000 ดอลต่อปี), ผมแดง, ขับ Tesla Model S และไปคอนเสิร์ต 3 ครั้งจริง

# code สำหรับเรียกข้อมูลมาดูไม่เกี่ยวกับสอบครับอันนี้

In [13]:
import pandas as pd

# ปรับให้ตารางแสดงอ่านง่ายขึ้นใน Colab
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

BASE = "/content/drive/MyDrive/midthem"

paths = {
    "crime_scene_report": f"{BASE}/crime_scene_report.csv",
    "person": f"{BASE}/person.csv",
    "drivers_license": f"{BASE}/drivers_license.csv",
    "interview": f"{BASE}/interview.csv",
    "get_fit_now_member": f"{BASE}/get_fit_now_member.csv",
    "get_fit_now_check_in": f"{BASE}/get_fit_now_check_in.csv",
    "facebook_event_checkin": f"{BASE}/facebook_event_checkin.csv",
    "income": f"{BASE}/income.csv",
}

# ฟังก์ชันสั้น ๆ สำหรับพรีวิวข้อมูล
def preview(df: pd.DataFrame, name: str, n: int = 5):
    print(f"\n========== {name} ==========")
    print(f"shape: {df.shape}")
    print("\n[columns]")
    print(list(df.columns))
    print("\n[dtypes]")
    print(df.dtypes)
    print("\n[null counts]")
    print(df.isna().sum())
    print(f"\n[head({n})]")
    display(df.head(n))

# อ่านไฟล์ "แบบปกติ" (ให้ pandas เดา dtype เอง)
dfs = {}
for key, p in paths.items():
    dfs[key] = pd.read_csv(p)

# พรีวิวทุกไฟล์แบบรวดเดียว
for key, df in dfs.items():
    preview(df, key, n=5)

# ----------------------------
# ตัวช่วยเสริม (เลือกใช้ได้)
# ----------------------------

# 1) ดูค่า unique เบื้องต้นของคอลัมน์สำคัญ (ปรับแก้รายการตามใจ)
important_unique = {
    "crime_scene_report": ["date", "type", "city"],
    "person": ["id", "name", "license_id", "address_street_name"],
    "drivers_license": ["id", "gender", "hair_color", "eye_color", "car_make", "car_model"],
    "interview": ["person_id"],
    "get_fit_now_member": ["id", "person_id", "membership_status"],
    "get_fit_now_check_in": ["membership_id", "check_in_date"],
    "facebook_event_checkin": ["event_name", "date"],
    "income": ["annual_income"],
}

print("\n\n========== quick unique peek ==========")
for key, cols in important_unique.items():
    df = dfs[key]
    print(f"\n--- {key} ---")
    for c in cols:
        if c in df.columns:
            # แสดงจำนวน unique + ตัวอย่างค่า
            uniq = df[c].dropna().unique()
            print(f"{c}: unique={len(uniq)}  sample={uniq[:5]}")
        else:
            print(f"{c}: (not found)")

# 2) ถ้าอยากดูสถิติพื้นฐานเฉพาะคอลัมน์ตัวเลขทั้งหมด
print("\n\n========== numeric describe (บางไฟล์อาจไม่มีคอลัมน์ตัวเลข) ==========")
for key, df in dfs.items():
    num = df.select_dtypes(include=["number"])
    if num.shape[1] > 0:
        print(f"\n--- {key} ---")
        display(num.describe().T)

# 3) ดูค่าที่พบบ่อยของบางคอลัมน์ (value_counts) เช่นประเภทคดี/ชื่ออีเวนต์
value_counts_wishlist = [
    ("crime_scene_report", "type"),
    ("crime_scene_report", "city"),
    ("facebook_event_checkin", "event_name"),
    ("get_fit_now_member", "membership_status"),
]
print("\n\n========== value_counts เลือกดู ==========")
for key, col in value_counts_wishlist:
    df = dfs[key]
    if col in df.columns:
        print(f"\n--- {key}.{col} (top 10) ---")
        display(df[col].value_counts().head(10))
    else:
        print(f"\n--- {key}.{col} --- (not found)")



========== crime_scene_report ==========
shape: (1228, 4)

[columns]
['date', 'type', 'description', 'city']

[dtypes]
date            int64
type           object
description    object
city           object
dtype: object

[null counts]
date           0
type           0
description    0
city           0
dtype: int64

[head(5)]


,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,Pandas City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,Pandas City



========== person ==========
shape: (10011, 6)

[columns]
['id', 'name', 'license_id', 'address_number', 'address_street_name', 'ssn']

[dtypes]
id                      int64
name                   object
license_id              int64
address_number          int64
address_street_name    object
ssn                     int64
dtype: object

[null counts]
id                     0
name                   0
license_id             0
address_number         0
address_street_name    0
ssn                    0
dtype: int64

[head(5)]


,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684



========== drivers_license ==========
shape: (10007, 9)

[columns]
['id', 'age', 'height', 'eye_color', 'hair_color', 'gender', 'plate_number', 'car_make', 'car_model']

[dtypes]
id               int64
age              int64
height           int64
eye_color       object
hair_color      object
gender          object
plate_number    object
car_make        object
car_model       object
dtype: object

[null counts]
id              0
age             0
height          0
eye_color       0
hair_color      0
gender          0
plate_number    0
car_make        0
car_model       0
dtype: int64

[head(5)]


...

,count
city,
Murfreesboro,9
Pandas City,9
Toledo,8
Jersey City,8
Evansville,8
Duluth,8
Little Rock,7
Lancaster,7
Wilmington,7



--- facebook_event_checkin.event_name (top 10) ---


,count
event_name,
\n,280
SQL Symphony Concert,212
-- Mark Twain\n,139
-- Oscar Wilde\n,103
We wish you a Hare Krishna\n,67
-- Kehlog Albran\n,67
-- Alan Perlis\n,60
"You patch a bug, and dump it again:\n",48
-- Albert Einstein\n,48



--- get_fit_now_member.membership_status (top 10) ---


,count
membership_status,
gold,68
silver,63
regular,53
